# Metadata

```yaml
course:   DS 5001 
module:   Final Project
topic:    Exploratory Text Analytics using Television Sitcom Scripts Final Report
author:   Eric Tria
date:     2 May 2023
```

# Student Info

```yaml
name:     Eric Tria
user_id:  emt4wf
email:    emt4wf@virginia.edu
```

# Project: Exploratory Text Analytics using Television Sitcom Scripts

## 1. Introduction
Describe the nature of your corpus and the question(s) you've asked of the data.

## 2. Source Data
Provide a description of all relativant source files and describe the following features for each source file:

### 2.1 Provenance
Where did they come from? Describe the website or other source and provide relevant URLs.

### 2.2 Location
Provide a link to the source files in UVA Box.

### 2.3 Description
What is the general subject matter of the corpus? How many observations are there? What is the average document length?

### 2.4 Format
A description of both the file formats of the source files, e.g., plaintext, XML, CSV, etc., and the internal structure where applicable. For - example, if XML then specify document type (e.g., TEI or XHTML).

## 3. Data Model
Describe the analytical tables you generated in the process of tokenization, annotation, and analysis of your corpus. You provide a list of tables with field names and their definition, along with URLs to each associated CSV file.

## 4. Exploration
Describe each of your explorations, such as PCA and topic models. For each, include the relevant parameters and hyperparemeters used to generate each model and visualization. For your visualizations, you should use at least three (but likely more) of the following visualization types:

### 4.1 Hierarchical cluster diagrams

### 4.2 Heatmaps showing correlations

### 4.3 Scatter plots

### 4.4 KDE plots

### 4.5 Dispersion plots

### 4.6 t-SNE plots

## 5. Interpretation
Provide your interpretation of the results of exploration, and any conclusion if you are comfortable making them.